**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Dropout

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns an action in function of the state the agent is in. The epsilon is here to sometimes select randomly an action to perform independently of the state. Its goal is to explore to get some more knowledge about which actions do what, to be able to learn better actions to perform. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw, outputdict={"-pix_fmt":"yuv420p"})

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_test=5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Position is used to position the player in the grid, the current state would have the position vector (array of size grid_sizexgrid_size transformed in vector of shape grid_size²x1) which corresponds to the position of the player before the move. Board contains has the shape grid_sizexgrid_size and contains the bonus (cheese) and the malus (poison). It contains only 0 except on cells with bonus or malus, which are positive or negative. Once the a bonus - or a malus - is taken the cell has the value 0 on board. There's a padding for the position of 2, filled with -1

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(4)
        

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    #loss = 0
    for e in range(epochs):
        state = env.reset()
        win = 0
        lose = 0
        game_over = False
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
    
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.0/12.0. Average score (-1.0)
Win/lose count 7.0/20.0. Average score (-7.0)
Win/lose count 10.0/12.0. Average score (-5.333333333333333)
Win/lose count 10.5/14.0. Average score (-4.875)
Win/lose count 8.5/12.0. Average score (-4.6)
Final score: -4.6


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}





1. We have :
\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a].\\
Q^\pi(s,a)=E_{p^{\pi}}[r(s, a) + \gamma\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a].\\
Q^\pi(s,a)=r(s, a) + \gamma E_{s'\sim \pi^*(.|s,a)}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a'].\\
Q^\pi(s,a)=r(s, a) + \gamma E_{s'\sim \pi^*(.|s,a)}[Q^{\pi}(s',a')].
\end{equation*}
We deduce:
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')].
\end{equation*}


2. For all $a'$ which doesn't satisfy $\max_{a'}Q(s',a') = Q(s',a')$  we have :

\begin{equation*}
\max_{a'}Q(s',a') > Q(s',a')\\
E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma \max_{a'}Q(s',a')] > E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q(s',a')]\\
\end{equation*}
So if according to a certain policy $\pi$ we take such $a'$ as an action, then $\pi$ isn't optimal. Hence we have: 
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

3. Since we have :

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]\\
E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] - Q^{*}(s,a) = 0\\
E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')- Q^{*}(s,a)] = 0
\end{equation*}

We conclude that for a network $Q(s,a,\theta)$ which we want to be equal to $Q^*(s,a)$, a necesary condition is to have:

\begin{equation*}
E_{s' \sim \pi^*(.|s,a)}[r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)] = 0 \text{ (1)}
\end{equation*}

And since $E[X]^2 \leq E[X^2]$ (Jensen inequality), if $Q(s,a,\theta)$ statisfies :

\begin{equation*}
E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2} = 0
\end{equation*}

Then it would satisfy (1).

So we could take as a loss for that network :

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory: # if the memory isn't full
            self.memory.append(m)
        else: # if it's full replace one of the sequence
            self.memory[np.random.randint(0, len(self.memory))] = m 

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))] # pick randomly one of the sequences

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, axis=0))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access() # take one of the sequences
            input_states[i] = s_ # add the state as input
            target_a = self.model.predict(np.expand_dims(n_s_, axis=0))[0] # compute Q(n_s, a) for each a
            value = self.model.predict(np.expand_dims(s_, axis=0))[0] # compute Q(s, a) for each a
            target_q[i] = np.array(value) # fill the target as Q(s, a) for each a
            best_a = np.argmax(target_a) # take the best action for Q(n_s, a)
            # replace the Q(s, a) only for the action that has been taken
            # to make the other differences equal to 0, which would make the loss be equal to the formula above
            if game_over_: 
                target_q[i, a_] = r_ # if the game is over we earn only r 
            else:
                target_q[i, a_] = r_ + self.discount*target_a[best_a] # otherwise we earn r and the discounted future gain
                ######## FILL IN
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        model = Sequential()
        model.add(Reshape((25*self.n_state,)))
        model.add(Dense(80, activation='tanh'))
        model.add(Dense(20, activation='tanh'))
        model.add(Dense(4))
        # NN Model
        
        ####### FILL IN
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
epochs_train = 21
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))







Epoch 000/021 | Loss 0.0054 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/021 | Loss 0.0131 | Win/lose count 5.5/6.0 (-0.5)
Epoch 002/021 | Loss 0.0305 | Win/lose count 6.5/7.0 (-0.5)
Epoch 003/021 | Loss 0.0039 | Win/lose count 5.5/4.0 (1.5)
Epoch 004/021 | Loss 0.0036 | Win/lose count 2.0/5.0 (-3.0)
Epoch 005/021 | Loss 0.0088 | Win/lose count 4.5/2.0 (2.5)
Epoch 006/021 | Loss 0.0072 | Win/lose count 4.5/2.0 (2.5)
Epoch 007/021 | Loss 0.0056 | Win/lose count 4.5/2.0 (2.5)
Epoch 008/021 | Loss 0.0038 | Win/lose count 4.0/1.0 (3.0)
Epoch 009/021 | Loss 0.0037 | Win/lose count 10.0/2.0 (8.0)
Epoch 010/021 | Loss 0.0028 | Win/lose count 12.0/2.0 (10.0)
Epoch 011/021 | Loss 0.0040 | Win/lose count 4.0/0 (4.0)
Epoch 012/021 | Loss 0.0056 | Win/lose count 4.0/4.0 (0.0)
Epoch 013/021 | Loss 0.0030 | Win/lose count 11.5/3.0 (8.5)
Epoch 014/021 | Loss 0.0492 | Win/lose count 13.5/2.0 (11.5)
Epoch 015/021 | Loss 0.0054 | Win/lose count 11.5/4.0 (7.5)
Epoch 016/021 | Loss 0.0025 | Win/lose cou

The default learning rate batch size and memory size happened to be the best one to choose. A good criterium to stop the learning phase is when the algorithm score doesn't improve which corresponds to the situation where the loss starts oscillating (which is around 20 epochs).

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model = Sequential()
        model.add(Conv2D(80, (2, 2), activation='relu'))
        model.add(Conv2D(20, (2, 2), activation='relu'))
        model.add(keras.layers.Flatten())
        model.add(Dense(4))
        ###### FILL IN
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
epochs_train = 21
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

Epoch 000/021 | Loss 0.0465 | Win/lose count 5.0/5.0 (0.0)
Epoch 001/021 | Loss 0.0602 | Win/lose count 7.5/2.0 (5.5)
Epoch 002/021 | Loss 0.0014 | Win/lose count 9.0/0 (9.0)
Epoch 003/021 | Loss 0.0062 | Win/lose count 7.5/4.0 (3.5)
Epoch 004/021 | Loss 0.0035 | Win/lose count 12.5/7.0 (5.5)
Epoch 005/021 | Loss 0.0012 | Win/lose count 3.0/2.0 (1.0)
Epoch 006/021 | Loss 0.0043 | Win/lose count 7.5/7.0 (0.5)
Epoch 007/021 | Loss 0.0027 | Win/lose count 11.5/2.0 (9.5)
Epoch 008/021 | Loss 0.0033 | Win/lose count 5.5/3.0 (2.5)
Epoch 009/021 | Loss 0.0016 | Win/lose count 3.5/2.0 (1.5)
Epoch 010/021 | Loss 0.0018 | Win/lose count 6.5/4.0 (2.5)
Epoch 011/021 | Loss 0.0032 | Win/lose count 9.0/3.0 (6.0)
Epoch 012/021 | Loss 0.0035 | Win/lose count 4.0/4.0 (0.0)
Epoch 013/021 | Loss 0.0561 | Win/lose count 10.0/1.0 (9.0)
Epoch 014/021 | Loss 0.0066 | Win/lose count 5.5/0 (5.5)
Epoch 015/021 | Loss 0.0261 | Win/lose count 13.0/4.0 (9.0)
Epoch 016/021 | Loss 0.0022 | Win/lose count 5.5/3.0 (2.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
epochs_test = 11
test(agent_cnn,env,epochs_test, prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test, prefix='fc_test')

Test of the CNN
Win/lose count 8.0/2.0. Average score (6.0)
Win/lose count 5.0/1.0. Average score (5.0)
Win/lose count 7.0/3.0. Average score (4.666666666666667)
Win/lose count 6.0/1.0. Average score (4.75)
Win/lose count 3.0/2.0. Average score (4.0)
Win/lose count 11.5/0. Average score (5.25)
Win/lose count 5.5/3.0. Average score (4.857142857142857)
Win/lose count 6.0/3.0. Average score (4.625)
Win/lose count 5.0/0. Average score (4.666666666666667)
Win/lose count 8.5/2.0. Average score (4.85)
Win/lose count 7.5/3.0. Average score (4.818181818181818)
Final score: 4.818181818181818
Test of the FC
Win/lose count 4.0/10.0. Average score (-6.0)
Win/lose count 2.0/1.0. Average score (-2.5)
Win/lose count 4.5/4.0. Average score (-1.5)
Win/lose count 4.0/5.0. Average score (-1.375)
Win/lose count 3.0/8.0. Average score (-2.1)
Win/lose count 6.0/4.0. Average score (-1.4166666666666667)
Win/lose count 2.5/1.0. Average score (-1.0)
Win/lose count 7.5/4.0. Average score (-0.4375)
Win/lose count 

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

We can notice that the CNN works way better than the FCN. The biggest issue we can observe is that the algorithm get at some point stuck in a loop. Another problem is that the algorithm sometimes doesn't take the decision to take a bonus which is in his view range. Increasing the temperature increases the scores in general, which means that the algorithm understood that it has to take the bonuses over the maluses.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            #In train explore:
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')



In [19]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw, outputdict={"-pix_fmt":"yuv420p"})

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        
        

        
        self.t = self.t + 1
 

        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position  = np.copy(self.position)
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),
                        axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore20.mp4'))

Epoch 000/021 | Loss 0.0384 | Win/lose count 6.5/25.60000000000008 (-19.10000000000008)
Epoch 001/021 | Loss 0.0081 | Win/lose count 6.0/19.700000000000014 (-13.700000000000014)
Epoch 002/021 | Loss 0.0077 | Win/lose count 10.5/19.100000000000016 (-8.600000000000016)
Epoch 003/021 | Loss 0.0415 | Win/lose count 8.5/18.199999999999996 (-9.699999999999996)
Epoch 004/021 | Loss 0.0110 | Win/lose count 6.5/19.700000000000017 (-13.200000000000017)
Epoch 005/021 | Loss 0.0063 | Win/lose count 14.5/16.899999999999974 (-2.3999999999999737)
Epoch 006/021 | Loss 0.0068 | Win/lose count 15.5/22.299999999999986 (-6.7999999999999865)
Epoch 007/021 | Loss 0.0060 | Win/lose count 13.5/13.899999999999968 (-0.3999999999999684)
Epoch 008/021 | Loss 0.0103 | Win/lose count 20.5/14.299999999999965 (6.200000000000035)
Epoch 009/021 | Loss 0.0335 | Win/lose count 20.0/14.69999999999997 (5.300000000000029)
Epoch 010/021 | Loss 0.0229 | Win/lose count 14.0/19.300000000000015 (-5.300000000000015)
Epoch 011/021

In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 5.5/2.0. Average score (3.5)
Win/lose count 13.0/2.0. Average score (7.25)
Win/lose count 8.0/3.0. Average score (6.5)
Win/lose count 14.5/3.0. Average score (7.75)
Win/lose count 9.0/4.0. Average score (7.2)
Win/lose count 19.5/4.0. Average score (8.583333333333334)
Win/lose count 10.5/6.0. Average score (8.0)
Win/lose count 15.5/3.0. Average score (8.5625)
Win/lose count 19.5/2.0. Average score (9.555555555555555)
Win/lose count 6.5/2.0. Average score (9.05)
Win/lose count 14.5/3.0. Average score (9.272727272727273)
Final score: 9.272727272727273


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [22]:
def train_with_expert(agent, expert_agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The expert agent performs an action perform the action now
            action = expert_agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            #In train explore:
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')



In [23]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
expert_agent = agent
learner_agent = DQN_CNN(size, lr=.01, epsilon=0.1, memory_size=2000, batch_size=32, n_state=3)
train_with_expert(learner_agent, expert_agent, env, 21, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore20.mp4'))

Epoch 000/021 | Loss 0.0035 | Win/lose count 8.0/19.700000000000003 (-11.700000000000003)
Epoch 001/021 | Loss 0.0072 | Win/lose count 13.0/18.69999999999998 (-5.6999999999999815)
Epoch 002/021 | Loss 0.0015 | Win/lose count 14.0/17.599999999999973 (-3.599999999999973)
Epoch 003/021 | Loss 0.0058 | Win/lose count 19.0/12.799999999999978 (6.200000000000022)
Epoch 004/021 | Loss 0.0052 | Win/lose count 18.0/15.099999999999971 (2.9000000000000288)
Epoch 005/021 | Loss 0.0070 | Win/lose count 18.0/16.399999999999977 (1.6000000000000227)
Epoch 006/021 | Loss 0.0058 | Win/lose count 11.5/15.399999999999965 (-3.899999999999965)
Epoch 007/021 | Loss 0.0038 | Win/lose count 16.0/17.89999999999999 (-1.8999999999999915)
Epoch 008/021 | Loss 0.0038 | Win/lose count 5.0/18.699999999999996 (-13.699999999999996)
Epoch 009/021 | Loss 0.0073 | Win/lose count 14.0/17.199999999999985 (-3.199999999999985)
Epoch 010/021 | Loss 0.0083 | Win/lose count 17.5/16.79999999999998 (0.7000000000000206)
Epoch 011/02

In [24]:
test(learner_agent,env,epochs_test,prefix='cnn_test_expert')
HTML(display_videos('cnn_test_expert10.mp4'))

Win/lose count 8.5/1.0. Average score (7.5)
Win/lose count 13.0/4.0. Average score (8.25)
Win/lose count 11.5/2.0. Average score (8.666666666666666)
Win/lose count 19.5/2.0. Average score (10.875)
Win/lose count 12.5/2.0. Average score (10.8)
Win/lose count 12.5/5.0. Average score (10.25)
Win/lose count 12.0/8.0. Average score (9.357142857142858)
Win/lose count 13.0/4.0. Average score (9.3125)
Win/lose count 12.0/6.0. Average score (8.944444444444445)
Win/lose count 16.5/4.0. Average score (9.3)
Win/lose count 16.5/4.0. Average score (9.590909090909092)
Final score: 9.590909090909092


The model that mimicks the behavior has equivalent performances than the one starting from zero (which takes its own "best actions" to explore the map.)

***